In [3]:
import torch
import random
import sys
sys.path.append('/content/drive/MyDrive/Simple_RLHF')

from google.colab import drive

drive.mount('/content/drive')

from util import TokenizerUtil
tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

(tensor([   0, 9178,   32,    2]), tensor([1, 1, 1, 1]), '<s>how are</s>')

In [4]:
!pip install datasets
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='/content/drive/MyDrive/Simple_RLHF/dataset/train.json', split='train')

#2,4,4切分,取第0部分
dataset = dataset.select(range(15000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=2,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

(7500,
 {'input_ids': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]),
  'labels': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]])})

In [5]:
from google.colab import drive

drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Simple_RLHF')

from transformers import AutoModelForCausalLM
import costom_lora

model_actor = AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b')

costom_lora.insert(model_actor)
costom_lora.count_params(model_actor)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

{'count_require': 2.21044736, 'count_all': 14.29004288, 'ratio': 0.15468444556549854}


In [6]:
import torch

from transformers import get_scheduler
from accelerate import Accelerator


def f():
    params = []
    params_lora = []
    for name, param in model_actor.named_parameters():
        if not param.requires_grad:
            continue

        if 'lora_A' in name or 'lora_B' in name:
            params_lora.append(param)
            continue

        params.append(param)

    return [{
        'params': params,
        'weight_decay': 0.0,
    }, {
        'params': params_lora,
        'weight_decay': 0.0,
        'lr': 5e-4
    }]


optimizer = torch.optim.Adam(f(), lr=1e-3, betas=(0.9, 0.95))

scheduler = get_scheduler(name='cosine',
                          optimizer=optimizer,
                          num_warmup_steps=0,
                          num_training_steps=100)

accelerator = Accelerator(gradient_accumulation_steps=64,
                          mixed_precision='fp16')

model_actor, loader, optimizer, scheduler = accelerator.prepare(
    model_actor, loader, optimizer, scheduler)

model_actor.train()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (v_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (q_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (out_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)


In [7]:
for i, data in enumerate(loader):
    with accelerator.accumulate(model_actor):
        out = model_actor(**data)
        accelerator.backward(out.loss)

        if accelerator.sync_gradients:
            accelerator.clip_grad_norm_(
                [i for i in model_actor.parameters() if i.requires_grad], 1.0)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    if (i + 1) % 100 == 0:
        lr = optimizer.param_groups[0]['lr']
        print(i, len(loader), out.loss.item(), lr)

        logits = out.logits[0].argmax(1)
        print(tokenizer.decode(logits))

    if i == 2000:
        break

costom_lora.merge(model_actor)
model_actor.save_pretrained('/content/drive/MyDrive/Simple_RLHF/model/actor')

99 7500 11.980487823486328 0.0009997532801828658
oitarian
_�ATE TABLE (_name (of ( name_ARCHAR(
_ARCHAR, score VARCHAR, WHERE_ CRE is the? player player? by the?=? player???
: CRE player FROM table_name_20 WHERE place = tt8" AND player = "byronnelson" GROUPI the the the the a the the a the) a the the the the</s>
199 7500 2.909801959991455 0.00099778098230154
of)
 =�ATE TABLE "_name (of (
_ARCHAR ( name_ARCHAR) WHERE_SELECTs about name of theotin_aulz's to date dateIST( table_NAME_60 where VNERING_ "NicICOLAS"EREIRA" andI_<pad>_<pad>_<pad>__<pad><pad>__<pad>a_<pad><pad><pad>==<pad><pad>_=<pad>_<pad><pad>__os___<pad>_)_<pad>_<pad><pad>_<pad><pad><pad>=<pad>os_<pad>os<pad>_<pad><pad><pad><pad><pad><pad>_<pad>=<pad><pad><pad>_<pad>_=<pad><pad>_<pad>__<pad>=<pad><pad>__<pad>___<pad><pad>___<pad>_<pad>_<pad><pad>_=<pad><pad>____<pad><pad>____<pad>_<pad><pad><pad><pad>____<pad><pad><pad>_<pad>__.a<pad><pad><pad>=_<pad><pad>__a_<pad>_<pad>)<pad>)<pad><pad>_<pad><pad>os____<pad>___<pad><pad>__<

In [9]:
!ls -al /content/drive/MyDrive/Simple_RLHF/model/actor

total 5139760
-rw------- 1 root root        749 Mar  9 01:55 config.json
-rw------- 1 root root        132 Mar  9 01:55 generation_config.json
-rw------- 1 root root 4994509120 Mar  9 01:56 model-00001-of-00002.safetensors
-rw------- 1 root root  268568856 Mar  9 01:56 model-00002-of-00002.safetensors
-rw------- 1 root root      33832 Mar  9 01:56 model.safetensors.index.json
